In [1]:
import numpy as np
import pandas as pd 
import os
import csv
import random
import string
import nltk
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize
import torch
from transformers import BertTokenizer, BertForSequenceClassification
from torch.utils.data import DataLoader, TensorDataset

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: l

In [2]:
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        if '.csv' in filename:
            data_path = os.path.join(dirname, filename)
        else:
            stopword_path = '/kaggle/input/stopwords/stopwords/english'

In [3]:
print(stopword_path, data_path)
data = pd.read_csv(data_path, encoding='ISO-8859-1', names=["sentiment", "ids", "date", "flag", "user", "text"])
data = data[['sentiment', 'text']]
data = data.sample(frac= 0.0001, random_state= 69)
X, Y = data['text'], data['sentiment']
Y = Y.replace(4, 1)
print(data.head(10))

/kaggle/input/stopwords/stopwords/english /kaggle/input/sentiment140/training.1600000.processed.noemoticon.csv
         sentiment                                               text
331760           0  @michellebranch oh my!!!  I hope the videoshoo...
438241           0                                   Woke up to pain 
364313           0  @StayClassySon you're soo lucky! no fair, i st...
179622           0  1st- TY to those who follow back.  2nd - grrr ...
1514041          4  @BunnyBridget  who cares u could eat a million...
932925           4  Maita: chillin in CH's house in between shoots...
730618           0  tool in Ubunutu for creating effects with wind...
468854           0  @The_cobra666 @opinion8ed_dyke  yeah but this ...
225427           0  @ba1L33 If they get it... still a few financin...
1537939          4                        wotsits are the boom diggy 


In [4]:
print(f'dataset contains {data.shape[0]} rows')
Y.value_counts(normalize=True)

dataset contains 160 rows


1    0.5375
0    0.4625
Name: sentiment, dtype: float64

In [5]:
with open(stopword_path) as f:
    stopwords_list= f.readlines()
    stopwords = []
    for i in range(len(stopwords_list)):
        a = stopwords_list[i]
        b = a.strip('\n').lower()
        stopwords.append(b)

extras = ['your', 'u', 'my']
stopwords.extend(extras)
print(stopwords)
print(X.head(10))

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', 'her', 'hers', 'herself', 'it', 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', 'too', 'very', 's', 't', 'can', 'will', 'just', 'don', 'should', 'no

In [6]:
special_characters = list(string.punctuation)

nltk.download('punkt')
stemmer = PorterStemmer()

def remove_stopwords(text):
    text_wostemming = ''
    text_words = text.lower()
    text_words = text_words.split()
    sentiment_words = [word for word in text_words if word not in stopwords]
    sentiment_text = ' '.join(sentiment_words)
    for i in sentiment_text:
        if i in special_characters:
            continue
        else:
            text_wostemming = text_wostemming + i
    words = word_tokenize(text_wostemming)
    stemmed_words = [stemmer.stem(i) for i in words]
    stemmed_text = ' '.join(stemmed_words)
    return stemmed_text

X = X.apply(remove_stopwords)

[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [7]:
print(data.columns)
print(data.shape)
print(X)

Index(['sentiment', 'text'], dtype='object')
(160, 2)
331760     michellebranch oh my hope videoshoot kickass then
438241                                             woke pain
364313        stayclassyson your soo lucki fair still 2 week
179622     1st ty follow back 2nd grrr dont cuz mess abl ...
1514041    bunnybridget care could eat million still look...
                                 ...                        
1358596    sbeecreat lol oddli enough dont eat much lobst...
1209105    markhoppu declar republican prop8 get 0 follow...
1185147    gmanandrizk ye love itbad busi peepsbut travel...
951098     lilyrose74 lol give happi till 830 work today ...
110018                               jamash that noth happen
Name: text, Length: 160, dtype: object


In [8]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly i

In [9]:
input_ids = []
attention_masks = []

for text in data['text']:
    inputs = tokenizer.encode_plus(
        text,
        add_special_tokens=True,
        max_length=128,  # Adjust as needed
        padding='max_length',
        return_attention_mask=True,
        return_tensors='pt'
    )
    input_ids.append(inputs['input_ids'])
    attention_masks.append(inputs['attention_mask'])

input_ids = torch.cat(input_ids, dim=0)
attention_masks = torch.cat(attention_masks, dim=0)

labels = Y.to_numpy()
labels = torch.tensor(labels)

# Print the converted tensors
print("Input IDs:", input_ids.shape)
print("Attention Masks:", attention_masks.shape)
print("Labels:", labels)

Input IDs: torch.Size([160, 128])
Attention Masks: torch.Size([160, 128])
Labels: tensor([0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1,
        0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1, 0,
        1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1,
        1, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0,
        1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 1, 1,
        1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0,
        0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 1, 1, 1, 1, 0])


In [10]:
train_dataset = TensorDataset(input_ids, attention_masks, labels)
train_dataloader = DataLoader(train_dataset, batch_size=5, shuffle=True)

optimizer = torch.optim.AdamW(model.parameters(), lr=1e-5)
loss_fn = torch.nn.CrossEntropyLoss()

In [11]:
num_epochs = 5
for epoch in range(num_epochs):
    model.train()
    for batch in train_dataloader:
        optimizer.zero_grad()
        input_ids, attention_masks, labels = batch
        outputs = model(input_ids, attention_mask=attention_masks)[0]
        loss = loss_fn(outputs, labels)
        loss.backward()
        optimizer.step()